In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import xlrd
import config

In [2]:
dbname = config.dbname
user = config.user
password = config.password
host = config.host
port = config.port

In [3]:
data_file_path = r'D:\EVE_DB_BACKUP\raw_data\reference_data\mapRegions.csv'
df = pd.read_csv(data_file_path)
df['factionID'] = df['factionID'].fillna(0).astype(int)

In [5]:
print(df)

     regionID          regionName  factionID  nebula
0    10000001             Derelik     500007   11799
1    10000002           The Forge     500001   11806
2    10000003  Vale of the Silent          0   11814
3    10000004              UUA-F4          0   11817
4    10000005             Detorid          0   11849
..        ...                 ...        ...     ...
108  14000001               VR-01          0   11821
109  14000002               VR-02          0   11821
110  14000003               VR-03          0   11821
111  14000004               VR-04          0   11821
112  14000005               VR-05          0   11821

[113 rows x 4 columns]


In [6]:
def create_table(dbname, user, password, host, port, table_name):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    print("Check 2")
    # Create table if it doesn't exist
    cur.execute(sql.SQL("""
        CREATE TABLE IF NOT EXISTS {}
        (
            
            region_id INTEGER PRIMARY KEY NOT NULL,
            region_name VARCHAR(100) NOT NULL,
            faction_id INTEGER,
            nebula INTEGER

        );
    """).format(sql.Identifier(table_name)))
    # Commit changes and close connection
    conn.commit()
    cur.close()
    conn.close()

In [7]:
create_table(dbname, user, password, host, port, 'map_regions')

Check 2


In [8]:
def add_timepoint(dbname, user, password, host, port, data, table_name):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    # Insert data
    print(data.head())
    for _, row in data.iterrows():
        cur.execute(sql.SQL("""
            INSERT INTO {} (region_id, region_name, faction_id, nebula) 
            VALUES (%s, %s, %s, %s);
        """).format(sql.Identifier(table_name)), 
            (row['regionID'], row['regionName'], row['factionID'], row['nebula']))
    # Commit changes and close connection
    conn.commit()
    cur.close()
    conn.close()

In [12]:
add_timepoint(dbname, user, password, host, port, df, 'map_regions')


   regionID          regionName  factionID  nebula
0  10000001             Derelik     500007   11799
1  10000002           The Forge     500001   11806
2  10000003  Vale of the Silent          0   11814
3  10000004              UUA-F4          0   11817
4  10000005             Detorid          0   11849


In [33]:
print(type(data))

<class 'int'>


In [37]:
print(data[1])

(12000, datetime.date(2023, 1, 1), 12000, 12000, 5, 1448, 10000002, 19)
